In [347]:
import logging
import os
import sys
from collections import defaultdict
import yaml
import json
import os
import re
import logging
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from functools import partial
from rdflib import Graph, Literal, URIRef
from rdflib.namespace import RDF, RDFS
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import dycomutils as common_utils
from typing import List, Dict, Any, Optional, Set, Tuple, DefaultDict
import openai

sys.path.append("/home/desild/work/research/chatbs/v2")

from src.utils.helpers import setup_logger
from src.utils.parser import graph_query_to_sexpr, is_inv_rel, get_inv_rel, graph_query_to_sparql
from src.utils.kg import get_readable_relation, get_readable_class, get_non_literals, get_nodes_by_class, \
    get_reverse_relation, get_reverse_readable_relation, prune_graph_query, legal_class, legal_relation
from src.utils.arguments import Arguments
from src.utils.sparql import SPARQLUtil, get_freebase_label, get_freebase_literals_by_cls_rel, \
    get_freebase_entid_lbl_by_cls
from src.utils.maps import literal_map

from transformers import set_seed
from tqdm import tqdm

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

from src.explorer_updates import Explorer, ExecutableProgram
from src.utils.graph_manager import GraphManager, regex_add_strings

In [348]:
# --- 1. Setup & Configuration ---
ROOT_DIR = os.path.abspath("/home/desild/work/research/chatbs")
V2_DIR = os.path.join(ROOT_DIR, "v2")

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = logging.getLogger(__name__)

In [349]:
# Load .env file from the specified path
# R: load_dot_env("../ChatBS-NexGen/.env")
env_path = os.path.join(V2_DIR, ".env")
log.info(f"Loading .env file from: {env_path}")
load_dotenv(env_path)

# Load YAML config
# R: config <- yaml::read_yaml(...)
config_path = os.path.join(V2_DIR, "prov.config.yaml")
log.info(f"Loading config: {config_path}")
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Load JSON metadata
# R: ttl_metadata <- readLines("QGraph_metadata.json")
metadata_path = os.path.join(V2_DIR, "data/workflow/chatbs_sample_metadata.json")
log.info(f"Loading metadata: {metadata_path}")
with open(metadata_path, 'r') as f:
    ttl_metadata = json.load(f)

11/26/2025 03:41:46 - INFO - __main__ -   Loading .env file from: /home/desild/work/research/chatbs/v2/.env
11/26/2025 03:41:46 - INFO - __main__ -   Loading config: /home/desild/work/research/chatbs/v2/prov.config.yaml
11/26/2025 03:41:46 - INFO - __main__ -   Loading metadata: /home/desild/work/research/chatbs/v2/data/workflow/chatbs_sample_metadata.json


In [350]:
graph_manager = GraphManager(config, os.path.join(V2_DIR, "data/workflow/10_sample_graph/chatbs_sample.ttl"))
schema = common_utils.serialization.load_json(os.path.join(V2_DIR, "data/workflow/schema.json"))
definitions = {'class_definitions':schema['classes'], 'relation_definitions':{k:v["description"] for k,v in schema['relations'].items()}}

11/26/2025 03:41:46 - INFO - src.utils.graph_manager -   Initializing GraphManager...
11/26/2025 03:41:46 - INFO - src.utils.graph_manager -   Graph loaded with 4062 triples.


In [351]:
collect_results = {}

In [352]:
TEST_SPARQL = """PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?p ?o ?pe ?po
WHERE {
    # 1. Start with the properties of the target object
    <{obj_uri}> ?p ?o .
    
    # 2. Check if the object is a prov:Collection
    # This pattern only binds ?isCollection if <{obj_uri}> is a prov:Collection
    OPTIONAL {
        <{obj_uri}> rdf:type prov:Collection .
        BIND(TRUE AS ?isCollection)
    }
    
    OPTIONAL {
        FILTER (bound(?isCollection))
        <{obj_uri}> prov:hadMember ?member .
        ?member ?pe ?po .
    }
}"""

In [353]:
graph_manager.query(regex_add_strings(TEST_SPARQL, obj_uri="http://testwebsite/testProgram#Collection-id_20251125040021_410-kg_info"))

,p,o,pe,po
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/ns/prov#Collection,http://testwebsite/testDFColumn#ingredientCat,Tropical fruit@en^^<xsd:string>
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/ns/prov#Collection,http://testwebsite/testDFColumn#ingredientName,Avocado@en^^<xsd:string>
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/ns/prov#Collection,http://testwebsite/testDFColumn#sugarG,0.66@en^^<xsd:string>
3,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/ns/prov#Collection,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://purl.org/provone#Data
4,http://www.w3.org/ns/prov#hadMember,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientCat,Tropical fruit@en^^<xsd:string>
5,http://www.w3.org/ns/prov#hadMember,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientName,Avocado@en^^<xsd:string>
6,http://www.w3.org/ns/prov#hadMember,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#sugarG,0.66@en^^<xsd:string>
7,http://www.w3.org/ns/prov#hadMember,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://purl.org/provone#Data
8,http://www.w3.org/ns/prov#wasGeneratedBy,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientCat,Tropical fruit@en^^<xsd:string>
9,http://www.w3.org/ns/prov#wasGeneratedBy,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientName,Avocado@en^^<xsd:string>


Q1: How many unique experimental are there in this, this accounts to the number of unique identifies in the executions ?

In [354]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT (count(distinct ?ids) AS ?obj_count)
WHERE {
     ?obj a provone:Execution ;
          dcterms:identifier ?ids .  
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "How many test ran through this ?",
     'results':ques_info.to_dict(orient='records')
}

ques_info

,obj_count
0,9


Q2: What are the AI Tasks that generated programs?

In [355]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?obj ?obj2
WHERE {
     ?obj a eo:AITask ;
          sio:SIO_000229 ?obj2 .
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "What are the AI Tasks that generated programs?",
     'results':ques_info["obj"].to_list(),
     "entity":ques_info["obj"].to_list(),
     "tags": ["structural"],
     "answer":"The AI Tasks that generated programs are listed in the results."
}

ques_info

,obj,obj2
0,http://testwebsite/testProgram#AI_Task-DF_combine,http://testwebsite/testProgram#DF_combine
1,http://testwebsite/testProgram#AI_Task-Pipeline,http://testwebsite/testProgram#Pipeline
2,http://testwebsite/testProgram#AI_Task-batch_s...,http://testwebsite/testProgram#batch_sparql_qu...
3,http://testwebsite/testProgram#AI_Task-informa...,http://testwebsite/testProgram#information_ext...
4,http://testwebsite/testProgram#AI_Task-llm_chat,http://testwebsite/testProgram#llm_chat
5,http://testwebsite/testProgram#AI_Task-query_r...,http://testwebsite/testProgram#query_result_po...
6,http://testwebsite/testProgram#AI_Task-sparql_...,http://testwebsite/testProgram#sparql_query_ex...
7,http://testwebsite/testProgram#AI_Task-system_...,http://testwebsite/testProgram#system_prompt_g...


Q3: How many overall steps in this pipeline?

In [356]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct (count(?obj) as ?obj_count)
WHERE {
     ?obj a provone:Channel.
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "How many overall steps in this pipeline?",
     'results':ques_info.to_dict(orient='records')
}

ques_info

,obj_count
0,4


Q4: How many overall program executions ?

In [357]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct (count(?obj) as ?obj_count)
WHERE {
     ?obj a provone:Execution.
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "How many overall program executions ?",
     'results':ques_info.to_dict(orient='records')
}

ques_info

,obj_count
0,119


Q6: what was the SPARQL query used as an object record to the sparql_query_extraction AI Task?

In [358]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?param ?value
WHERE {
     <http://testwebsite/testProgram#AI_Task-sparql_query_extractor> sio:SIO_000230 ?param .
     ?param prov:value ?value
     Filter(contains(str(?value), "PREFIX"))
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what was the SPARQL query used as an object record to the sparql_query_extraction AI Task?",
     'results':ques_info["value"].to_list(),
     "entity":ques_info["param"].to_list(),
     "tags": ["row-level"],
     "answer":"The SPARQL query used as an object record to the sparql_query_extraction AI Task is listed in the results."
}

ques_info

,param,value
0,http://testwebsite/testProgram#AI_Task-sparql_...,PREFIX dbo: <http://dbpedia.org/ontology/>\nPR...


Q7: what are the instructions used as an object record to the post-processing-query step AI Task?

In [359]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?param ?value
WHERE {
     <http://testwebsite/testProgram#AI_Task-query_result_post_processor> sio:SIO_000230 ?param .
     ?param prov:value ?value
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what are the instructions used as an object record to the post-processing-query step AI Task?",
     'results':ques_info["value"].to_list(),
     "entity":ques_info["param"].to_list(),
     "tags": ["structural"],
     "answer":"The instructions used as an object record to the post-processing-query step AI Task are listed in the results."
}

ques_info

,param,value
0,http://testwebsite/testProgram#AI_Task-query_r...,Concat all the rows into single row at each co...
1,http://testwebsite/testProgram#AI_Task-query_r...,If there are duplicate ingredients then select...


Q8: what are the instructions used as an object record to the 'Information-Extraction' AI Task?

In [360]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?param ?value
WHERE {
     <http://testwebsite/testProgram#AI_Task-information_extractor> sio:SIO_000230 ?param .
     ?param prov:value ?value
     
     FILTER(contains(str(?value), "Extract all"))
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what are the instructions used as an object record to the 'Information-Extraction' AI Task?",
     'results':ques_info["value"].to_list(),
     "entity":ques_info["param"].to_list(),
     "tags": ["structural"],
     "answer":"The instructions used as an object record to the 'Information-Extraction' AI Task are listed in the results."
}

ques_info

,param,value
0,http://testwebsite/testProgram#AI_Task-informa...,Extract all the main ingredients without any s...


Q9: what was the builder llm used as an object record to the system_prompt_template AI Task?

In [361]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?param ?value
WHERE {
     <http://testwebsite/testProgram#AI_Task-system_prompt_generator> sio:SIO_000230 ?param .
     ?param prov:value ?value
     
     Filter(contains(str(?value), "gpt"))
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what was the builder llm used as an object record to the system_prompt_template AI Task?",
     'results':ques_info["value"].to_list(),
     "entity":ques_info["param"].to_list(),
     "tags": ["structural"],
     "answer":"The instructions used as an object record to the 'Information-Extraction' AI Task are listed in the results."
}

ques_info

,param,value
0,http://testwebsite/testProgram#AI_Task-system_...,gpt-4o@en^^<xsd:string>


Q10-2: what was the builder llm used as an object record to the sparql_query_extraction AI Task?

In [362]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?param ?value
WHERE {
     <http://testwebsite/testProgram#AI_Task-sparql_query_extractor> sio:SIO_000230 ?param .
     ?param prov:value ?value
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)


collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what was the builder llm used as an object record to the sparql_query_extraction AI Task?",
     'results':[],
     "entity":[],
     "tags": ["structural"],
     "answer":"There is no builder LLM for the sparql_query_extraction AI Task. as it is not build using a generative model."
}

ques_info

,param,value
0,http://testwebsite/testProgram#AI_Task-sparql_...,https://dbpedia.org/sparql@en^^<xsd:string>
1,http://testwebsite/testProgram#AI_Task-sparql_...,PREFIX dbo: <http://dbpedia.org/ontology/>\nPR...


Q10: what was the builder llm used as an object record to the information-extraction AI Task?

In [363]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?param ?value
WHERE {
     <http://testwebsite/testProgram#AI_Task-information_extractor> sio:SIO_000230 ?param .
     ?param prov:value ?value
     
     Filter(contains(str(?value), "gpt"))
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)


collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what was the builder llm used as an object record to the information-extraction AI Task?",
     'results':ques_info["value"].to_list(),
     "entity":ques_info["param"].to_list(),
     "tags": ["structural"],
     "answer":"The instructions used as an object record to the 'Information-Extraction' AI Task are listed in the results."
}

ques_info

,param,value
0,http://testwebsite/testProgram#AI_Task-informa...,gpt-4o@en^^<xsd:string>


Q11: what is the steps in order of programs? [Extremely hard to answer at the current stage]

In [364]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?prog ?id ?label
WHERE {
     ?prog dcterms:identifier ?id ;
           rdfs:label ?label .
}
order by ?id
"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what is the steps in order of programs?",
     'results':ques_info["label"].to_list(),
     "entity":ques_info["prog"].to_list() + ques_info["label"].to_list(),
     "answer":"The steps in order of programs are listed in the results.",
     "tags": ["structural"],

}

ques_info

,prog,id,label
0,http://testwebsite/testProgram#system_prompt_g...,1@en^^<xsd:string>,System Prompt Template Generator@en^^<xsd:string>
1,http://testwebsite/testProgram#information_ext...,2@en^^<xsd:string>,Information Extractor@en^^<xsd:string>
2,http://testwebsite/testProgram#DF_combine,3@en^^<xsd:string>,Data Frame Combiner@en^^<xsd:string>
3,http://testwebsite/testProgram#batch_sparql_qu...,3@en^^<xsd:string>,Batch SPARQL Query Extractor@en^^<xsd:string>
4,http://testwebsite/testProgram#sparql_query_ex...,3@en^^<xsd:string>,SPARQL Query Builder@en^^<xsd:string>
5,http://testwebsite/testProgram#query_result_po...,4@en^^<xsd:string>,Query Result Post-Processor@en^^<xsd:string>


Q12-2 what is the program that takes the system prompt created by the program system_prompt_generator as an input ?

In [365]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?value ?label
WHERE {
     <http://testwebsite/testProgram#system_prompt_generator> provone:hasOutPort/provone:connectsTo ?channel .
     ?value provone:hasInPort/provone:connectsTo ?channel .
     ?value rdfs:label ?label .
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what is the program that takes the system prompt created by the program system_prompt_generator as an input ?",
     'results':ques_info["label"].to_list(),
     "entity":ques_info["value"].to_list(),
     "tags": ["structural"],
     "answer":"The next program that is fed the output of the system_prompt_generator is LLM Chat"
}

ques_info

,value,label
0,http://testwebsite/testProgram#llm_chat,LLM Chat@en^^<xsd:string>


what is the program that uses as input the output of the llm_chat program?

In [366]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?value ?label
WHERE {
     <http://testwebsite/testProgram#llm_chat> provone:hasOutPort/provone:connectsTo ?channel .
     ?value provone:hasInPort/provone:connectsTo ?channel .
     ?value rdfs:label ?label .
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what is the program that uses as input the output of the llm_chat program?",
     'results':ques_info["label"].to_list(),
     "entity":ques_info["value"].to_list(),
     "tags": ["structural"],
     "answer":"The next program that is fed the output of the llm_chat is the Information Extractor"
}

ques_info

,value,label
0,http://testwebsite/testProgram#information_ext...,Information Extractor@en^^<xsd:string>


Q12: what is the step after getting the LLM output

In [367]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?value
WHERE {
     <http://testwebsite/testProgram#llm_chat> provone:hasOutPort/provone:connectsTo ?channel .
     ?value provone:hasInPort/provone:connectsTo ?channel .
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what is the step after getting the LLM output ?",
     'results':ques_info.to_dict(orient='records'),
}

ques_info

,value
0,http://testwebsite/testProgram#information_ext...


Q13: what was the system prompt generated by the execution of system_prompt_generation with id 1_2

In [368]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?data ?sys_temp
WHERE {
     ?exe dcterms:identifier "1_2@en^^<xsd:string>" ;
          prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#system_prompt_generator> .
          
     ?data prov:wasGeneratedBy ?exe .
     ?data prov:value ?sys_temp .
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what was the system prompt generated by the execution of system_prompt_generation with id 1_2",
     'results':ques_info["sys_temp"].to_list(),
     "entity":ques_info["data"].to_list(),
     "tags": ["row-level"],
     "answer":"The system prompt generated by the execution of system_prompt_generation with id 1_2 is listed in the results."
}


ques_info

,data,sys_temp
0,http://testwebsite/testProgram#Data-id_2025112...,You are a culinary assistant that recommends o...


Q13:  what was the sparql results generated by the execution of sparql_query_extractor with id 1_2

In [369]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?d ?key ?value
WHERE {
     ?exe dcterms:identifier "1_2@en^^<xsd:string>" ;
          prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#sparql_query_extractor> .
          
     ?c prov:wasGeneratedBy ?exe .
     ?c prov:hadMember ?d.
     ?d ?key ?value .
     
     FILTER (?key != rdf:type && ?key != prov:hadMember)
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what was the sparql results generated by the execution of sparql_query_extractor with id 1_2",
     'results':ques_info["value"].to_list(),
     "entity":ques_info["d"].to_list(),
     "tags": ["row-level"],
     "answer":"The sparql results generated by the execution of sparql_query_extractor with id 1_2 are listed in the results."

}

ques_info

,d,key,value
0,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientCat,NA@en^^<xsd:string>
1,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientName,NA@en^^<xsd:string>
2,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
3,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientCat,@en^^<xsd:string>
4,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientName,@en^^<xsd:string>
5,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#sugarG,@en^^<xsd:string>
6,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientCat,@en^^<xsd:string>
7,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientName,@en^^<xsd:string>
8,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#sugarG,@en^^<xsd:string>
9,http://testwebsite/testProgram#Data-id_2025112...,http://testwebsite/testDFColumn#ingredientCat,NA@en^^<xsd:string>


Q14:  what were the extracted ingredients generated by the execution of information_extraction with id 1_2

In [370]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?d ?key ?value
WHERE {
     ?exe dcterms:identifier "1_2@en^^<xsd:string>" ;
          prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#information_extractor> .
          
     ?c prov:wasGeneratedBy ?exe .
     ?c prov:hadMember ?d.
     ?d ?key ?value .
     
     FILTER (?key != rdf:type && ?key != prov:hadMember)
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what were the extracted ingredients generated by the execution of information_extraction with id 1_2",
     'results':ques_info["value"].to_list(),
     "entity":ques_info["d"].to_list(),
     "tags": ["row-level"],
     "answer":"The extracted ingredients generated by the execution of information_extractor with id 1_2 are listed in the results."

}

ques_info

,d,key,value
0,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Salt@en^^<xsd:string>
1,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Pepper@en^^<xsd:string>
2,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Olive oil@en^^<xsd:string>
3,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Egg@en^^<xsd:string>
4,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Spinach@en^^<xsd:string>
5,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Feta cheese@en^^<xsd:string>
6,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Turkey bacon@en^^<xsd:string>
7,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Parsley@en^^<xsd:string>


Q15:  what were the extracted ingridients from the dbpedia that had no value retrieved for execution 1_2

In [371]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?c2 ?exe ?out_key ?out_value
WHERE {
     ?exe dcterms:identifier "1_2@en^^<xsd:string>" ;
          prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#sparql_query_extractor> .
          
     ?c1 prov:wasGeneratedBy ?exe .
     ?exe prov:used/prov:value ?c2 .
     ?c1 prov:hadMember ?d.
     ?d ?out_key ?out_value .
     
     FILTER (?out_key != rdf:type && ?out_key != prov:hadMember)
     
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what were the extracted ingridients from the dbpedia that had no value retrieved for execution 1_2",
     'results':ques_info.to_dict(orient='records')
}

ques_info

,c2,exe,out_key,out_value
0,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientCat,NA@en^^<xsd:string>
1,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientName,NA@en^^<xsd:string>
2,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
3,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientCat,@en^^<xsd:string>
4,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientName,@en^^<xsd:string>
5,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,@en^^<xsd:string>
6,Pepper@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientCat,@en^^<xsd:string>
7,Pepper@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientName,@en^^<xsd:string>
8,Pepper@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,@en^^<xsd:string>
9,Pepper@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#ingredientCat,NA@en^^<xsd:string>


Q16:  what are the LLM suggested ingredients that were above 1 grams of sugar for experiment with id 1_2

In [372]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?c2 ?exe ?out_key ?floatOut
WHERE {
     ?exe dcterms:identifier "1_2@en^^<xsd:string>" ;
          prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#sparql_query_extractor> .
          
     ?c1 prov:wasGeneratedBy ?exe .
     ?exe prov:used/prov:value ?c2 .
     ?c1 prov:hadMember ?d.
     ?d ?out_key ?out_value .
     
     FILTER (?out_key != rdf:type && ?out_key != prov:hadMember)
     BIND (xsd:float(STRBEFORE(?out_value, "@")) AS ?floatOut)
     
     FILTER (?floatOut > 1.0)
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)

collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what are the LLM suggested ingredients that were above 1 grams of sugar for experiment with id 1_2",
     'results':ques_info.to_dict(orient='records')
}

ques_info

,c2,exe,out_key,floatOut


Q17:  what are the ingredients that were suggested by the LLM during the experiment ?

In [373]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?d ?key ?value
WHERE {
     ?exe prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#information_extractor> .
          
     ?c prov:wasGeneratedBy ?exe .
     ?c prov:hadMember ?d.
     ?d ?key ?value .
     
     FILTER (?key != rdf:type && ?key != prov:hadMember)
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)


collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what are the ingredients that were suggested by the LLM during the experiment ?",
     'results':ques_info.to_dict(orient='records')
}


ques_info

,d,key,value
0,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Salt@en^^<xsd:string>
1,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Olive Oil@en^^<xsd:string>
2,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Egg@en^^<xsd:string>
3,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Pepper@en^^<xsd:string>
4,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Avocado@en^^<xsd:string>
...,...,...,...
78,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Salt@en^^<xsd:string>
79,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Pepper@en^^<xsd:string>
80,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Spinach@en^^<xsd:string>
81,http://testwebsite/testProgram#Data-id_2025112...,http://www.w3.org/ns/prov#value,Avocado@en^^<xsd:string>


Q19:  what are the most common ingredients suggested to diabetic patients?

In [374]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT ?dout (COUNT(?dout) AS ?dout_count)
WHERE {
     ?exe prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#information_extractor> .
          
     ?c prov:wasGeneratedBy ?exe .
     ?c prov:hadMember/prov:value ?dout.
     
     ?prev_exe prov:wasInformedBy/prov:wasInformedBy  ?exe;
               prov:qualifiedUsage ?usage.
     
     ?usage provone:hadEntity/prov:value ?din .
     ?usage provone:hadInPort ?in_port .
     FILTER (?in_port = <http://testwebsite/testProgram#system_prompt_generator-diagnosis>)
     Filter (?din = "Diabetic@en^^<xsd:string>")
}
GROUP BY ?dout
order by DESC(?dout_count)
"""

ques_info = graph_manager.query(QUESTION_SPARQL)



collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what are the most common ingredients suggested to diabetic patients?",
     'results':ques_info.to_dict(orient='records')
}


ques_info

,dout,dout_count
0,Egg@en^^<xsd:string>,4
1,Spinach@en^^<xsd:string>,4
2,Salt@en^^<xsd:string>,3
3,Pepper@en^^<xsd:string>,3
4,Olive oil@en^^<xsd:string>,2
5,Garlic@en^^<xsd:string>,2
6,Mushroom@en^^<xsd:string>,2
7,Feta cheese@en^^<xsd:string>,1
8,Turkey bacon@en^^<xsd:string>,1
9,Parsley@en^^<xsd:string>,1


Q20: what are the ingredients that were suggested by the LLM during the experiment that has values sugar values in the knowledge graph?

In [375]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?c2 ?exe ?out_key ?out_value
WHERE {
     ?exe prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#sparql_query_extractor> .
          
     ?c1 prov:wasGeneratedBy ?exe .
     ?exe prov:used/prov:value ?c2 .
     ?c1 prov:hadMember ?d.
     ?d ?out_key ?out_value .
     
     FILTER (CONTAINS(LCASE(str(?out_key)), "sugar"))
     FILTER (!CONTAINS(LCASE(str(?out_value)), "na")  && ?out_value != "@en^^<xsd:string>")
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)


collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what are the ingredients that were suggested by the LLM during the experiment that has values sugar values in the knowledge graph?",
     'results':ques_info.to_dict(orient='records')
}


ques_info

,c2,exe,out_key,out_value
0,Avocado@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,0.66@en^^<xsd:string>
1,Parsley@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,0.85@en^^<xsd:string>
2,Spinach@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,0.4@en^^<xsd:string>
3,Spinach@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250401...,http://testwebsite/testDFColumn#sugarG,0.4@en^^<xsd:string>
4,Onion@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250401...,http://testwebsite/testDFColumn#sugarG,4.24@en^^<xsd:string>
5,Parsley@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250401...,http://testwebsite/testDFColumn#sugarG,0.85@en^^<xsd:string>
6,Honey@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250401...,http://testwebsite/testDFColumn#sugarG,82@en^^<xsd:string>
7,Coconut@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250401...,http://testwebsite/testDFColumn#sugarG,6.23@en^^<xsd:string>
8,Spinach@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250402...,http://testwebsite/testDFColumn#sugarG,0.4@en^^<xsd:string>
9,Onion@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250402...,http://testwebsite/testDFColumn#sugarG,4.24@en^^<xsd:string>


Q21: what are the ingredients that were suggested by the LLM during the experiment that has no value in the knowledge graph?

In [376]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT distinct ?c2 ?exe ?out_key ?out_value
WHERE {
     ?exe prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#sparql_query_extractor> .
          
     ?c1 prov:wasGeneratedBy ?exe .
     ?exe prov:used/prov:value ?c2 .
     ?c1 prov:hadMember ?d.
     ?d ?out_key ?out_value .
     
     FILTER (CONTAINS(LCASE(str(?out_key)), "sugar"))
     FILTER (CONTAINS(LCASE(str(?out_value)), "na")  || ?out_value = "@en^^<xsd:string>")
}

"""

ques_info = graph_manager.query(QUESTION_SPARQL)


collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what are the ingredients that were suggested by the LLM during the experiment that has no value in the knowledge graph?",
     'results':ques_info.to_dict(orient='records')
}


ques_info

,c2,exe,out_key,out_value
0,Berry@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
1,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,@en^^<xsd:string>
2,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
3,Salt@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
4,Pepper@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250400...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
...,...,...,...,...
94,Mushroom@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250404...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
95,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250404...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>
96,Egg@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250404...,http://testwebsite/testDFColumn#sugarG,@en^^<xsd:string>
97,Pepper@en^^<xsd:string>,http://testwebsite/testProgram#id_202511250404...,http://testwebsite/testDFColumn#sugarG,NA@en^^<xsd:string>


Q22: what is the highest and average sugar ingredient suggested in the experientation?.

In [377]:
QUESTION_SPARQL = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT ?din (count(?floatOut) AS ?countFloatOut) (AVG(?floatOut) AS ?avgFloatOut) (MIN(?floatOut) AS ?minFloatOut) (MAX(?floatOut) AS ?maxFloatOut)
WHERE {
     ?exe prov:qualifiedAssociation/prov:hadPlan <http://testwebsite/testProgram#sparql_query_extractor> .
     
     ?prev_exe prov:wasInformedBy ?exe.
     ?prev_exe2 prov:wasInformedBy ?prev_exe.
     
     ?prev_exe3 prov:wasInformedBy ?prev_exe2;
                prov:qualifiedUsage ?usage.
     
     ?usage provone:hadEntity/prov:value ?din .
     ?usage provone:hadInPort ?in_port .
     FILTER (?in_port = <http://testwebsite/testProgram#system_prompt_generator-diagnosis>)
     
     ?c prov:wasGeneratedBy ?exe .
     ?c prov:hadMember ?dobj .
     ?dobj ?p ?dout .
     
     FILTER (CONTAINS(LCASE(str(?p)), "sugar"))
     FILTER (!CONTAINS(LCASE(str(?dout)), "na")  && ?dout != "@en^^<xsd:string>")
     
     BIND (xsd:float(STRBEFORE(?dout, "@")) AS ?floatOut)
}
group by ?din
order by DESC(?countFloatOut)
"""


ques_info = graph_manager.query(QUESTION_SPARQL)



collect_results[len(collect_results)] = {
     'sparql_query':QUESTION_SPARQL,
     'question': "what is the highest and average sugar ingredient suggested in the experientation?",
     'results':ques_info.to_dict(orient='records')
}


ques_info

,din,countFloatOut,avgFloatOut,minFloatOut,maxFloatOut
0,Diabetic@en^^<xsd:string>,10,0.9410000000000001,0.06,4.24
1,None@en^^<xsd:string>,8,1.4325,0.4,4.24
2,Hypertensive@en^^<xsd:string>,5,18.826,0.66,82.0


In [378]:
os.makedirs('evalutions/GT', exist_ok=True)
common_utils.serialization.save_json(collect_results, 
                                     os.path.join("evalutions", "GT", "questions_results.json")
                                     )